In [1]:
import requests
import os
import pandas as pd
import json
from dateutil import parser
from dotenv import load_dotenv

In [2]:
load_dotenv()
private_api = os.getenv("API_KEY")
private_account = os.getenv("ACCOUNT_ID")
private_url = os.getenv("OANDA_URL")

In [3]:
API_KEY = private_api
ACCOUNT_ID = private_account
OANDA_URL = private_url

In [4]:
session = requests.Session()

In [5]:
session.headers.update({
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
})

In [6]:
params = dict(
    count = 10,
    granularity = "H1",
    price = "MBA"
)

In [7]:
url = f"{OANDA_URL}/accounts/{ACCOUNT_ID}/instruments"

In [8]:
response = session.get(url, params=None, data=None, headers=None)

In [9]:
response.status_code

200

In [10]:
data = response.json()

In [11]:
instruments_list = data['instruments']

In [12]:
len(instruments_list)

68

In [13]:
instruments_list[0].keys()

dict_keys(['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'minimumTradeSize', 'maximumTrailingStopDistance', 'minimumTrailingStopDistance', 'maximumPositionSize', 'maximumOrderUnits', 'marginRate', 'guaranteedStopLossOrderMode', 'tags', 'financing'])

In [14]:
key_inst = ['name', 'type', 'displayName', 'pipLocation', 'displayPrecision', 'tradeUnitsPrecision', 'marginRate']

In [15]:
instruments_dict = {}

for i in instruments_list:
    key = i['name']
    instruments_dict[key] = { k: i[k] for k in key_inst}

In [16]:
instruments_dict['USD_CAD']

{'name': 'USD_CAD',
 'type': 'CURRENCY',
 'displayName': 'USD/CAD',
 'pipLocation': -4,
 'displayPrecision': 5,
 'tradeUnitsPrecision': 0,
 'marginRate': '0.02'}

In [17]:
# -4 -> 0.0001
pow(10, -4)

0.0001

In [18]:
with open("../data/instruments.json", "w") as f:
    f.write(json.dumps(instruments_dict, indent=2))

In [19]:
def fetch_candles(pair_name, count=10, granularity="H1"):
    url = f"{OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
        )
    response = session.get(url, params=params, data=None, headers=None)
    data = response.json()

    if response.status_code == 200:
        if "candles" not in data:
            data = []
        else: 
            data = data['candles']
    return response.status_code, data

def get_candles_df(data):
    if len(data) == 0:
        return pd.DataFrame()
    
    prices = ["mid", "bid", "ask"]
    ohlc = ["o", "h", "l", "c"]

    final_data = []
    for candle in data:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = parser.parse(candle['time'])
        new_dict['volume'] = candle['volume']

        for p in prices:
            for o in ohlc:
                new_dict[f'{p}_{o}'] = float(candle[p][o])
        
        final_data.append(new_dict)

    df = pd.DataFrame.from_dict(final_data)
    return df

def create_data_file(pair_name, count=10, granularity="H1"):
    code, data = fetch_candles(pair_name, count, granularity)
    if code != 200:
        print("Failed", pair_name, data)
        return
    
    if len(data) == 0:
        print("no candles", pair_name)

    candles_df = get_candles_df(data)
    candles_df.to_pickle(f"../data/{pair_name}_{granularity}.pkl")
    print(f"{pair_name} {granularity} {candles_df.shape[0]} candles, {candles_df.time.min()} {candles_df.time.max()}")


In [20]:
code, data = fetch_candles("EUR_USD", count=10, granularity="H4")
candles_df = get_candles_df(data)

In [21]:
create_data_file("EUR_USD", count=10, granularity="H4")

EUR_USD H4 10 candles, 2023-08-31 05:00:00+00:00 2023-09-01 17:00:00+00:00


In [22]:
our_curr = ['EUR', 'USD', 'GBP', 'JPY', 'CHF', 'NZD', 'CAD', 'AUD']

In [23]:
instruments_dict.keys()

dict_keys(['USD_SGD', 'EUR_SEK', 'HKD_JPY', 'AUD_USD', 'USD_CAD', 'NZD_USD', 'NZD_SGD', 'USD_NOK', 'USD_CNH', 'SGD_CHF', 'GBP_JPY', 'USD_TRY', 'AUD_JPY', 'ZAR_JPY', 'SGD_JPY', 'GBP_ZAR', 'USD_JPY', 'EUR_TRY', 'EUR_JPY', 'AUD_SGD', 'EUR_NZD', 'GBP_HKD', 'CHF_JPY', 'EUR_HKD', 'GBP_CAD', 'USD_THB', 'GBP_CHF', 'AUD_CHF', 'NZD_CHF', 'AUD_HKD', 'USD_CHF', 'CAD_HKD', 'EUR_CHF', 'EUR_SGD', 'NZD_CAD', 'GBP_AUD', 'USD_PLN', 'EUR_ZAR', 'TRY_JPY', 'EUR_AUD', 'USD_ZAR', 'CAD_JPY', 'NZD_HKD', 'USD_CZK', 'USD_DKK', 'USD_SEK', 'GBP_SGD', 'EUR_DKK', 'CHF_ZAR', 'CAD_CHF', 'GBP_USD', 'USD_MXN', 'USD_HUF', 'USD_HKD', 'EUR_USD', 'EUR_CAD', 'AUD_CAD', 'GBP_PLN', 'EUR_PLN', 'GBP_NZD', 'EUR_HUF', 'EUR_NOK', 'CHF_HKD', 'EUR_GBP', 'AUD_NZD', 'CAD_SGD', 'EUR_CZK', 'NZD_JPY'])

In [25]:
for p1 in our_curr:
    for p2 in our_curr:
        pr = f"{p1}_{p2}"
        if pr in instruments_dict:
            for g in ["H1", "H4"]:
                create_data_file(pr, count=4001, granularity=g)


EUR_USD H1 4001 candles, 2023-01-12 05:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_USD H4 4001 candles, 2021-02-10 02:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_GBP H1 4001 candles, 2023-01-12 05:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_GBP H4 4001 candles, 2021-02-10 02:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_JPY H1 4001 candles, 2023-01-12 05:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_JPY H4 4001 candles, 2021-02-09 18:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_CHF H1 4001 candles, 2023-01-12 05:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_CHF H4 4001 candles, 2021-02-10 02:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_NZD H1 4001 candles, 2023-01-12 04:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_NZD H4 4001 candles, 2021-02-10 02:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_CAD H1 4001 candles, 2023-01-12 05:00:00+00:00 2023-09-01 20:00:00+00:00
EUR_CAD H4 4001 candles, 2021-02-10 02:00:00+00:00 2023-09-01 17:00:00+00:00
EUR_AUD H1 4001 candles, 2023-01-12 04:00:00+00:00 2023-09-01 20:00:00+00:00